In [3]:
%load_ext autoreload
%autoreload 2

In [12]:
import pandas as pd
import copy
import pprint
from benchmark_nerfstudio_paper import ablations_table_rows
from collections import defaultdict

# methods sorted in the same order we want in the paper tables
methods = [row[0] for row in ablations_table_rows]

In [34]:
def csv_filename_to_results(csv_filename, num_decimals=2, num_rows_to_consider=None):
    # code to read the csv file
    # and return a dictionary

    df = pd.read_csv(csv_filename)

    column_titles = list(df.columns.values)

    # iterate as long as the value is not None
    if num_rows_to_consider is None:
        rows = list(df.values)
    else:
        rows = list(df.values)[:num_rows_to_consider]
    valid_row = list(df.values[-1])
    for row in rows:
        for i in range(len(row)):
            value = row[i]
            if not pd.isnull(value):
                valid_row[i] = value
    last_row = valid_row
    # last_row will be the row that's valid

    from collections import defaultdict
    results = defaultdict(dict)
    for column_title, last_row_value in zip(column_titles, last_row):
        if column_title == "Step" or "__MAX" in column_title or "__MIN" in column_title:
            continue
        name = column_title.split(" - ")[0]
        capture_name, method = name.split("_")
        metric = float(last_row_value)
        # format to have 2 decimal places
        if num_decimals == 2:
            metric = f"{metric:.2f}"
        elif num_decimals == 3:
            metric = f"{metric:.3f}"
        else:
            metric = f"{metric}"
        results[capture_name][method] = metric

    # import pprint
    # pprint.pprint(results)
    # print(len(results))

    results = dict(results)
    return results

In [44]:
# the csv filenames
# these are generated from exporting CSVs from wandb graphs
csv_filename_psrn = "/home/ethanweber/nerfactory/wandb-psnr.csv"
csv_filename_ssim = "/home/ethanweber/nerfactory/wandb-ssim.csv"
csv_filename_lpips = "/home/ethanweber/nerfactory/wandb-lpips.csv"
results_psrn = csv_filename_to_results(csv_filename_psrn, num_decimals=None)
results_ssim = csv_filename_to_results(csv_filename_ssim, num_decimals=None)
results_lpips = csv_filename_to_results(csv_filename_lpips, num_decimals=None)


csv_filename_psrn = "/home/ethanweber/nerfactory/wandb-jan9-psnr.csv"
csv_filename_ssim = "/home/ethanweber/nerfactory/wandb-jan9-ssim.csv"
csv_filename_lpips = "/home/ethanweber/nerfactory/wandb-jan9-lpips.csv"
results_psrn_2 = csv_filename_to_results(csv_filename_psrn, num_decimals=None)
results_ssim_2 = csv_filename_to_results(csv_filename_ssim, num_decimals=None)
results_lpips_2 = csv_filename_to_results(csv_filename_lpips, num_decimals=None)

# merge the results
for capture_name, results in results_psrn_2.items():
    results_psrn[capture_name].update(results)
for capture_name, results in results_ssim_2.items():
    results_ssim[capture_name].update(results)
for capture_name, results in results_lpips_2.items():
    results_lpips[capture_name].update(results)


In [45]:
def get_averages_from_results(results):
    averages = defaultdict(list)
    for capture_name, methods in results.items():
        for method, metric in methods.items():
            averages[method].append(float(metric))
    for method, metrics in averages.items():
        val = sum(metrics) / len(metrics)
        averages[method] = float(f"{val:.4f}")
    return averages

In [46]:
pprint.pprint(get_averages_from_results(results_lpips))

defaultdict(<class 'list'>,
            {'1-prop-network': 0.0668,
             'l2-contraction': 0.0598,
             'nerfacto': 0.0601,
             'no-contraction': 0.1308,
             'random-background-color': 0.0599,
             'shared-prop-network': 0.0605,
             'synthetic-on-real': 0.114,
             'w/o-app': 0.0622,
             'w/o-pose': 0.0614,
             'w/o-pose-app': 0.0636})


In [47]:
def generate_row(method, capture_names):
    # remove spaces from method
    method_text = method.replace("-", " ")
    string = f"{method_text}"
    for idx, capture_name in enumerate(capture_names):
        psnr = results_psrn[capture_name][method]
        ssim = results_ssim[capture_name][method]
        lpips = results_lpips[capture_name][method]
        if idx == len(capture_names) - 1:
            string += " & \\multicolumn{1}{r|}{" + f"{psnr} / {ssim} / {lpips}" + "}"
        else:
            string += " & \\multicolumn{1}{r}{" + f"{psnr} / {ssim} / {lpips}" + "}"
    return string

In [48]:
capture_names_row1 = [
    "Egypt",
    "person",
    "kitchen",
    "plane",
    "dozer"
]
capture_names_row2 = [
    "floating-tree",
    "aspen",
    "stump",
    "sculpture",
    "Giannini-Hall"
]

print(capture_names_row1)
for method in methods[:-1]:
    # print(generate_row(method, capture_names_row1) + ' \\\ \cline{1-1}')
    print(generate_row(method, capture_names_row1) + ' \\\ ')
print(generate_row(methods[-1], capture_names_row1) + ' \\\ \hline')

print()

print(capture_names_row2)
for method in methods[:-1]:
    # print(generate_row(method, capture_names_row2) + ' \\\ \cline{1-1}')
    print(generate_row(method, capture_names_row2) + ' \\\ ')
print(generate_row(methods[-1], capture_names_row2) + ' \\\ \hline')



['Egypt', 'person', 'kitchen', 'plane', 'dozer']
nerfacto & \multicolumn{1}{r}{22.714982986450195 / 0.8457087874412537 / 0.0387668460607528} & \multicolumn{1}{r}{26.58124351501465 / 0.8588576912879944 / 0.0272847861051559} & \multicolumn{1}{r}{21.02389335632324 / 0.8730602860450745 / 0.0538232773542404} & \multicolumn{1}{r}{23.30121612548828 / 0.8020055890083313 / 0.0444588512182235} & \multicolumn{1}{r|}{21.23827743530273 / 0.8753665089607239 / 0.0552346743643283} \\ 
w/o pose & \multicolumn{1}{r}{22.709941864013672 / 0.8510463237762451 / 0.0384077467024326} & \multicolumn{1}{r}{26.875864028930664 / 0.875615656375885 / 0.0267722699791193} & \multicolumn{1}{r}{20.83899688720703 / 0.8627068996429443 / 0.0561660341918468} & \multicolumn{1}{r}{23.130054473876957 / 0.8018836975097656 / 0.0470539852976799} & \multicolumn{1}{r|}{21.35797691345215 / 0.8832465410232544 / 0.0533054806292057} \\ 
w/o app & \multicolumn{1}{r}{23.50516891479492 / 0.8450330495834351 / 0.0371274687349796} & \multico

In [13]:
# mipnerf360 experiments

csv_filename_psrn = "/home/ethanweber/nerfactory/mipnerf360-nerfacto-psnr.csv"
csv_filename_ssim = "/home/ethanweber/nerfactory/mipnerf360-nerfacto-ssim.csv"
csv_filename_lpips = "/home/ethanweber/nerfactory/mipnerf360-nerfacto-lpips.csv"

csv_filename_psrn = "/home/ethanweber/nerfactory/mipnerf360-instant-ngp-psnr.csv"
csv_filename_ssim = "/home/ethanweber/nerfactory/mipnerf360-instant-ngp-ssim.csv"
csv_filename_lpips = "/home/ethanweber/nerfactory/mipnerf360-instant-ngp-lpips.csv"

results_psrn = csv_filename_to_results(csv_filename_psrn, num_rows_to_consider=1)
results_ssim = csv_filename_to_results(csv_filename_ssim, num_decimals=3, num_rows_to_consider=1)
results_lpips = csv_filename_to_results(csv_filename_lpips, num_decimals=3, num_rows_to_consider=1)

In [11]:
mipnerf360_nerfacto_capture_names = ["bicycle", "garden", "stump", "room", "counter", "kitchen", "bonsai"]

print("psnr")
s = [(key, results_psrn[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()

print("ssim")
s = [(key, results_ssim[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()

print("lpips")
s = [(key, results_lpips[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()



psnr
[('bicycle', {'instant-ngp-w/o-pose-app': '19.92'}),
 ('garden', {'instant-ngp-w/o-pose-app': '22.68'}),
 ('stump', {'instant-ngp-w/o-pose-app': '20.25'}),
 ('room', {'instant-ngp-w/o-pose-app': '28.04'}),
 ('counter', {'instant-ngp-w/o-pose-app': '23.59'}),
 ('kitchen', {'instant-ngp-w/o-pose-app': '24.61'}),
 ('bonsai', {'instant-ngp-w/o-pose-app': '25.67'})]

ssim
[('bicycle', {'instant-ngp-w/o-pose-app': '0.448'}),
 ('garden', {'instant-ngp-w/o-pose-app': '0.542'}),
 ('stump', {'instant-ngp-w/o-pose-app': '0.465'}),
 ('room', {'instant-ngp-w/o-pose-app': '0.828'}),
 ('counter', {'instant-ngp-w/o-pose-app': '0.701'}),
 ('kitchen', {'instant-ngp-w/o-pose-app': '0.642'}),
 ('bonsai', {'instant-ngp-w/o-pose-app': '0.792'})]

lpips
[('bicycle', {'instant-ngp-w/o-pose-app': '0.449'}),
 ('garden', {'instant-ngp-w/o-pose-app': '0.402'}),
 ('stump', {'instant-ngp-w/o-pose-app': '0.440'}),
 ('room', {'instant-ngp-w/o-pose-app': '0.170'}),
 ('counter', {'instant-ngp-w/o-pose-app': '0.271

In [14]:
mipnerf360_nerfacto_capture_names = ["bicycle", "garden", "stump", "room", "counter", "kitchen", "bonsai"]

print("psnr")
s = [(key, results_psrn[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()

print("ssim")
s = [(key, results_ssim[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()

print("lpips")
s = [(key, results_lpips[key]) for key in mipnerf360_nerfacto_capture_names]
pprint.pprint(s)
print()

psnr
[('bicycle', {'instant-ngp-w/o-pose-app': '18.78'}),
 ('garden', {'instant-ngp-w/o-pose-app': '20.82'}),
 ('stump', {'instant-ngp-w/o-pose-app': '19.73'}),
 ('room', {'instant-ngp-w/o-pose-app': '25.26'}),
 ('counter', {'instant-ngp-w/o-pose-app': '22.09'}),
 ('kitchen', {'instant-ngp-w/o-pose-app': '22.82'}),
 ('bonsai', {'instant-ngp-w/o-pose-app': '23.86'})]

ssim
[('bicycle', {'instant-ngp-w/o-pose-app': '0.389'}),
 ('garden', {'instant-ngp-w/o-pose-app': '0.390'}),
 ('stump', {'instant-ngp-w/o-pose-app': '0.408'}),
 ('room', {'instant-ngp-w/o-pose-app': '0.749'}),
 ('counter', {'instant-ngp-w/o-pose-app': '0.621'}),
 ('kitchen', {'instant-ngp-w/o-pose-app': '0.530'}),
 ('bonsai', {'instant-ngp-w/o-pose-app': '0.665'})]

lpips
[('bicycle', {'instant-ngp-w/o-pose-app': '0.620'}),
 ('garden', {'instant-ngp-w/o-pose-app': '0.619'}),
 ('stump', {'instant-ngp-w/o-pose-app': '0.607'}),
 ('room', {'instant-ngp-w/o-pose-app': '0.311'}),
 ('counter', {'instant-ngp-w/o-pose-app': '0.417